In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
tf.__version__

In [ ]:
keras.__version__

In [ ]:
model = keras.Sequential()
# 第一层是输入层，无需制定输入的shape，会自动匹配，也不需要activation？
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation="softmax"))

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
data = np.random.random((1000,32))
label = np.random.random((1000,10))
val_data = np.random.random((100, 32))
val_label = np.random.random((100, 10))

# 使用data/label训练模型

In [ ]:
history = model.fit(data, label, epochs=10, batch_size=100)

In [ ]:
model.fit(data, label, epochs=10, batch_size=32,
          validation_data=(val_data, val_label))

# 使用Dataset训练模型
直接使用data/label训练模型的时候，传入fit方法的data/label是完整数据集。使用Dataset训练模型时，传入fit的数据集必须经过batch/repeat处理。在使用Dataset训练模型时，所传入的steps_per_epoch参数意为：在每轮（epoch）训练中，每次喂入多少个批次的数据。因此，假设训练数据有1000行，每个批次100行，那么当steps_per_epoch=10时，一轮训练完毕正好使用完所有数据。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data,label))
print(dataset.output_shapes)
dataset = dataset.batch(100)
dataset = dataset.repeat()

#history = model.fit(dataset, epochs=10, steps_per_epoch=40)

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset.batch(32).repeat()

# 执行会报错，只要加上validation_data这个部分？
model.fit(dataset,epochs=10,steps_per_epoch=20,
         validation_data=val_dataset,
         validation_steps=3)

# 回调

In [ ]:
# 目前不能执行，报告 `CancelledError:  [Op:StatefulPartitionedCall]`
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, label, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_label))